# CLI Interface

> Command-line interface for docments compliance checking

In [ ]:
#| default_exp cli

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import argparse
import sys
import json
from pathlib import Path
from typing import Optional
from cjm_nbdev_docments.report import check_project, generate_text_report, generate_json_report
from nbdev.config import get_config

In [ ]:
#| export
def main(
    args: Optional[list] = None  # Command line arguments (for testing)
) -> int:  # Exit code (0 for success, 1 for non-compliance)
    "Main CLI entry point for docments checker"
    parser = argparse.ArgumentParser(
        description="Check nbdev project for docments compliance",
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog="""
Examples:
  # Check current project
  nbdev-docments
  
  # Check specific notebooks directory
  nbdev-docments --nbs-path ./notebooks
  
  # Generate JSON report
  nbdev-docments --format json
  
  # Save report to file
  nbdev-docments --output report.txt
  
  # Show all definitions (including compliant ones)
  nbdev-docments --verbose
  
  # Show only functions with TODO placeholders
  nbdev-docments --todos-only
  
  # Auto-fix non-compliant functions
  nbdev-docments --fix
  
  # Preview fixes without applying
  nbdev-docments --fix --dry-run
"""
    )
    
    parser.add_argument(
        "--nbs-path",
        type=Path,
        help="Path to notebooks directory (defaults to nbdev config)"
    )
    
    parser.add_argument(
        "--format",
        choices=["text", "json"],
        default="text",
        help="Output format (default: text)"
    )
    
    parser.add_argument(
        "--output", "-o",
        type=Path,
        help="Save report to file instead of printing"
    )
    
    parser.add_argument(
        "--verbose", "-v",
        action="store_true",
        help="Show compliant definitions in text report"
    )
    
    parser.add_argument(
        "--quiet", "-q",
        action="store_true",
        help="Only show summary (exit code indicates compliance)"
    )
    
    parser.add_argument(
        "--todos-only",
        action="store_true",
        help="Show only functions with TODO placeholders"
    )
    
    parser.add_argument(
        "--fix",
        action="store_true",
        help="Auto-fix non-compliant functions by adding placeholder docs"
    )
    
    parser.add_argument(
        "--dry-run",
        action="store_true",
        help="Show what would be fixed without making changes"
    )
    
    # Parse arguments
    parsed_args = parser.parse_args(args)
    
    # Handle auto-fix mode
    if parsed_args.fix:
        from cjm_nbdev_docments.autofix import fix_notebook
        
        if parsed_args.nbs_path:
            nbs_path = parsed_args.nbs_path
        else:
            cfg = get_config()
            nbs_path = Path(cfg.config_path) / cfg.nbs_path
        
        # Fix all notebooks
        total_fixed = 0
        for nb_path in nbs_path.glob("*.ipynb"):
            if not nb_path.name.startswith('_'):
                try:
                    changes = fix_notebook(nb_path, dry_run=parsed_args.dry_run)
                    total_fixed += len(changes['functions_fixed'])
                except Exception as e:
                    print(f"Error fixing {nb_path}: {e}", file=sys.stderr)
        
        return 0 if not parsed_args.dry_run else (1 if total_fixed > 0 else 0)
    
    # Normal check mode
    try:
        results = check_project(parsed_args.nbs_path)
    except Exception as e:
        print(f"Error: {e}", file=sys.stderr)
        return 1
    
    # Filter for TODOs only if requested
    if parsed_args.todos_only:
        results = [r for r in results if r.has_todos]
        if not results:
            print("✅ No functions with TODO placeholders found!")
            return 0
    
    # Generate report
    if parsed_args.format == "json":
        report = json.dumps(generate_json_report(results), indent=2)
    else:
        report = generate_text_report(results, verbose=parsed_args.verbose)
    
    # Output report
    if not parsed_args.quiet:
        if parsed_args.output:
            parsed_args.output.write_text(report)
            print(f"Report saved to {parsed_args.output}")
        else:
            print(report)
    
    # Return exit code based on compliance
    non_compliant = sum(1 for r in results if not r.is_compliant)
    return 0 if non_compliant == 0 else 1

In [ ]:
# Test the CLI with different options
print("Basic check:")
exit_code = main([])
print(f"\nExit code: {exit_code}")

print("\n" + "="*50 + "\n")

print("JSON format:")
main(["--format", "json"])

Basic check:
📚 Docments Compliance Report
Total definitions: 16
✅ Compliant: 16
❌ Non-compliant: 0
⚠️  With TODO placeholders: 2


⚠️  Definitions with TODO placeholders:
------------------------------

📓 00_core.ipynb:
  ⚠️  check_definition (1 TODOs)

📓 03_autofix.ipynb:
  ⚠️  generate_fixed_source (7 TODOs)

Exit code: 0


JSON format:
{
  "summary": {
    "total": 16,
    "compliant": 16,
    "non_compliant": 0,
    "with_todos": 2,
    "total_todos": 8
  },
  "by_notebook": {
    "00_core.ipynb": {
      "compliant": [
        {
          "name": "DocmentsCheckResult",
          "type": "ClassDef",
          "has_docstring": true,
          "missing_params": [],
          "params_documented": {},
          "return_documented": true,
          "has_todos": false,
          "todo_count": 0
        },
        {
          "name": "extract_param_docs",
          "type": "FunctionDef",
          "has_docstring": true,
          "missing_params": [],
          "params_documented": {
    

0

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()